In [18]:
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroquery.ipac.nexsci.nasa_exoplanet_archive import NasaExoplanetArchive
from geopy import geocoders  
from geopy.geocoders import Nominatim
from datetime import datetime

In [4]:
geolocator = Nominatim(user_agent='moeur')
location = geolocator.geocode("Long Beach California")


In [12]:
ra_dec = location[1]

In [13]:
ra_dec

(33.7690164, -118.191604)

In [14]:
def julian_date(date_time):
    jd = date_time.toordinal() + 1721424.5 + (date_time.hour + date_time.minute / 60 + date_time.second / 3600) / 24
    return jd

In [15]:
def gst_from_jd(jd):
    '''
    Calculate Greenwich Sidereal Time (GST) from Julian Date.
    '''
    T = (jd - 2451545.0) / 36525
    GST = 280.46061837 + 360.98564736629 * (jd - 2451545) + T**2 * (0.000387933 - T / 38710000)
    GST = GST % 360
    return GST

In [21]:
def lst_from_gst(gst, longitude):
    """
    Calculate Local Sidereal Time (LST) from Greenwich Sidereal Time (GST) and longitude.
    """
    LST = gst + longitude
    LST = LST % 360  # Normalize to 0 - 360 degrees
    return LST

In [22]:
def zenith_ra_dec(latitude, longitude, date_time):
    """
    Compute the Right Ascension (RA) and Declination (Dec) of the zenith for a given location and date.
    """
    jd = julian_date(date_time)
    gst = gst_from_jd(jd)
    lst = lst_from_gst(gst, longitude)
    dec = latitude
    ra = lst
    return ra, dec

In [23]:
date_time = datetime(1998, 9, 4, 17, 0, 0)# year, month, day, hour, min, sec

In [24]:
ra, dec = zenith_ra_dec(ra_dec[0], ra_dec[1], date_time)

In [27]:
test_table = NasaExoplanetArchive.query_region(
    table="ps", coordinates=SkyCoord(ra * u.deg, dec * u.deg),
    radius=10 * u.deg)

In [32]:
test_table[0]

pl_name,pl_letter,hostname,hd_name,hip_name,tic_id,gaia_id,default_flag,pl_refname,sy_refname,disc_pubdate,disc_year,discoverymethod,disc_locale,disc_facility,disc_instrument,disc_telescope,disc_refname,ra,rastr,dec,decstr,glon,glat,elon,elat,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,pl_orbperstr,pl_orblpererr1,pl_orblper,pl_orblpererr2,pl_orblperlim,pl_orblperstr,pl_orbsmax,pl_orbsmaxerr1,pl_orbsmaxerr2,pl_orbsmaxlim,pl_orbsmaxstr,pl_orbincl,pl_orbinclerr1,pl_orbinclerr2,pl_orbincllim,pl_orbinclstr,pl_orbtper,pl_orbtpererr1,pl_orbtpererr2,pl_orbtperlim,pl_orbtperstr,pl_orbeccen,pl_orbeccenerr1,pl_orbeccenerr2,pl_orbeccenlim,pl_orbeccenstr,pl_eqt,pl_eqterr1,pl_eqterr2,pl_eqtlim,pl_eqtstr,pl_occdep,pl_occdeperr1,pl_occdeperr2,pl_occdeplim,pl_occdepstr,pl_insol,pl_insolerr1,pl_insolerr2,pl_insollim,pl_insolstr,pl_dens,pl_denserr1,pl_denserr2,pl_denslim,pl_densstr,pl_trandep,pl_trandeperr1,pl_trandeperr2,pl_trandeplim,pl_trandepstr,pl_tranmid,pl_tranmiderr1,pl_tranmiderr2,pl_tranmidlim,pl_tranmidstr,pl_trandur,pl_trandurerr1,pl_trandurerr2,pl_trandurlim,pl_trandurstr,sy_kmagstr,sy_umag,sy_umagerr1,sy_umagerr2,sy_umagstr,sy_rmag,sy_rmagerr1,sy_rmagerr2,sy_rmagstr,sy_imag,sy_imagerr1,sy_imagerr2,sy_imagstr,sy_zmag,sy_zmagerr1,sy_zmagerr2,sy_zmagstr,sy_w1mag,sy_w1magerr1,sy_w1magerr2,sy_w1magstr,sy_w2mag,sy_w2magerr1,sy_w2magerr2,sy_w2magstr,sy_w3mag,sy_w3magerr1,sy_w3magerr2,sy_w3magstr,sy_w4mag,sy_w4magerr1,sy_w4magerr2,sy_w4magstr,sy_gmag,sy_gmagerr1,sy_gmagerr2,sy_gmagstr,sy_gaiamag,sy_gaiamagerr1,sy_gaiamagerr2,sy_gaiamagstr,sy_tmag,sy_tmagerr1,sy_tmagerr2,sy_tmagstr,pl_controv_flag,pl_tsystemref,st_metratio,st_spectype,sy_kepmag,sy_kepmagerr1,sy_kepmagerr2,sy_kepmagstr,st_rotp,st_rotperr1,st_rotperr2,st_rotplim,st_rotpstr,pl_projobliq,pl_projobliqerr1,pl_projobliqerr2,pl_projobliqlim,pl_projobliqstr,x,y,z,htm20,pl_rvamp,pl_rvamperr1,pl_rvamperr2,pl_rvamplim,pl_rvampstr,pl_radj,pl_radjerr1,pl_radjerr2,pl_radjlim,pl_radjstr,pl_rade,pl_radeerr1,pl_radeerr2,pl_radelim,pl_radestr,pl_ratror,pl_ratrorerr1,pl_ratrorerr2,pl_ratrorlim,pl_ratrorstr,pl_ratdor,pl_trueobliq,pl_trueobliqerr1,pl_trueobliqerr2,pl_trueobliqlim,pl_trueobliqstr,sy_icmag,sy_icmagerr1,sy_icmagerr2,sy_icmagstr,rowupdate,pl_pubdate,st_refname,releasedate,dkin_flag,pl_ratdorerr1,pl_ratdorerr2,pl_ratdorlim,pl_ratdorstr,pl_imppar,pl_impparerr1,pl_impparerr2,pl_impparlim,pl_impparstr,pl_cmassj,pl_cmassjerr1,pl_cmassjerr2,pl_cmassjlim,pl_cmassjstr,pl_cmasse,pl_cmasseerr1,pl_cmasseerr2,pl_cmasselim,pl_cmassestr,pl_massj,pl_massjerr1,pl_massjerr2,pl_massjlim,pl_massjstr,pl_masse,pl_masseerr1,pl_masseerr2,pl_masselim,pl_massestr,pl_bmassj,pl_bmassjerr1,pl_bmassjerr2,pl_bmassjlim,pl_bmassjstr,pl_bmasse,pl_bmasseerr1,pl_bmasseerr2,pl_bmasselim,pl_bmassestr,pl_bmassprov,pl_msinij,pl_msinijerr1,pl_msinijerr2,pl_msinijlim,pl_msinijstr,pl_msinie,pl_msinieerr1,pl_msinieerr2,pl_msinielim,pl_msiniestr,st_teff,st_tefferr1,st_tefferr2,st_tefflim,st_teffstr,st_met,st_meterr1,st_meterr2,st_metlim,st_metstr,st_radv,st_radverr1,st_radverr2,st_radvlim,st_radvstr,st_vsin,st_vsinerr1,st_vsinerr2,st_vsinlim,st_vsinstr,st_lum,st_lumerr1,st_lumerr2,st_lumlim,st_lumstr,st_logg,st_loggerr1,st_loggerr2,st_logglim,st_loggstr,st_age,st_ageerr1,st_ageerr2,st_agelim,st_agestr,st_mass,st_masserr1,st_masserr2,st_masslim,st_massstr,st_dens,st_denserr1,st_denserr2,st_denslim,st_densstr,st_rad,st_raderr1,st_raderr2,st_radlim,st_radstr,ttv_flag,ptv_flag,tran_flag,rv_flag,ast_flag,obm_flag,micro_flag,etv_flag,ima_flag,pul_flag,soltype,sy_snum,sy_pnum,sy_mnum,cb_flag,st_nphot,st_nrvc,st_nspec,pl_nespec,pl_ntranspec,pl_nnotes,sy_pm,sy_pmerr1,sy_pmerr2,sy_pmstr,sy_pmra,sy_pmraerr1,sy_pmraerr2,sy_pmrastr,sy_pmdec,sy_pmdecerr1,sy_pmdecerr2,sy_pmdecstr,sy_plx,sy_plxerr1,sy_plxerr2,sy_plxstr,sy_dist,sy_disterr1,sy_disterr2,sy_diststr,sy_bmag,sy_bmagerr1,sy_bmagerr2,sy_bmagstr,sy_vmag,sy_vmagerr1,sy_vmagerr2,sy_vmagstr,sy_jmag,sy_jmagerr1,sy_jmagerr2,sy_jmagstr,sy_hmag,sy_hmagerr1,sy_hmagerr2,sy_hmagstr,sy_k

In [30]:
test_table[0]["hostname"]

'HD 62509'

In [66]:
def map_eccentricity_to_trait(eccentricity):
    if eccentricity < 0.3:
        return "like stability"
    elif 0.3 <= eccentricity < 0.6:
        return "are balanced"
    else:
        return "like change"

def map_semi_major_axis_to_trait(semi_major_axis):
    if semi_major_axis < 0.5:  
        return "introverted"
    else:
        return "extroverted"

In [31]:
def map_period_to_trait(period):
    if period < 10:
        return "quick thinker"
    elif 10 <= period < 100:
        return "balanced thinker"
    else:
        return "deep thinker"

def map_stellar_mass_to_trait(stellar_mass):
    if stellar_mass < 1.0:
        return "modest"
    elif 1.0 <= stellar_mass < 1.5:
        return "confident"
    else:
        return "assertive"

In [69]:
def generate_horoscope_message(table):
    name = table[0]['pl_name']
    star = table[0]['hostname']
    eccentricity = table[0]["pl_orbeccen"]
    semi_major_axis = float(table[0]['pl_orbsmax'].value)
    period = float(table[0]['pl_orbper'].value)
    stellar_mass = float(table[0]['st_mass'].value)
    
    eccentricity_trait = map_eccentricity_to_trait(eccentricity)
    axis_trait = map_semi_major_axis_to_trait(semi_major_axis)
    period_trait = map_period_to_trait(period)
    stellar_mass_trait = map_stellar_mass_to_trait(stellar_mass)
    
    message = (f"Your birth exoplanet is {name} orbiting star {star}. "
               f"Based on an eccentricity of {eccentricity:.2f}, you {eccentricity_trait}. "
               f"With an orbit semi-major axis of {semi_major_axis:.2f} AU, you are {axis_trait}. "
               f"Your exoplanet period of {period:.2f} days makes you a {period_trait}, "
               f"and with a stellar mass of {stellar_mass:.2f} solar masses, you are {stellar_mass_trait}.")
    
    return message

In [68]:
generate_horoscope_message(test_table)

'Your birth exoplanet is HD 62509 b orbiting star HD 62509. Based on an eccentricity of 0.06, you like stability. With an orbit semi-major axis of 1.69 AU, you are extroverted. Your period of 589.70 days makes you a deep thinker, and with a stellar mass of 1.70 solar masses, you are assertive.'